This notebook is a test notebook from the NIGGMS ME-INBRE project that will run an RNAseq workflow.

In [14]:
# Import necessary packages
import subprocess as sp
import os

In [6]:
# Test out the following commands. First, set up your directory structure and download files with this command. 
# You can peek at what it is doing by typing cat scripts/setup.sh.
# Note that since this is Python script we have to wrap our bash commands in supprocess calls
shell_call = "sh scripts/setup.sh"
proc_shell = sp.call(shell_call,shell=True)


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7175k  100 7175k    0     0  6619k      0  0:00:01  0:00:01 --:--:-- 6625k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7175k  100 7175k    0     0  7841k      0 --:--:-- --:--:-- --:--:-- 7833k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7178k  100 7178k    0     0  9052k      0 --:--:-- --:--:-- --:--:-- 9052k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7178k  100 7178k    0     0  9860k      0 --:--:-- --:--:-- --:--:-- 9846k
  % Total    % Received % Xferd  Average Speed   Tim

In [27]:
# Everything should now be downloaded
# Let's trim our raw reads
# First make a list of all files we want to trim (here we only have two)
files = [i.strip() for i in open("scripts/samples")]
# Then create a loop for each file
# use 'f' string to insert the {f} variable for each sample
for f in files:
    trim_call = f"trimmomatic PE -threads 2 data/raw_fastq/{f}_1.fastq data/raw_fastq/{f}_2.fastq data/trimmed/{f}_trimmed_1.fastq data/trimmed/{f}_trimmed_2.fastq data/trimmed/{f}_trimmed_2_unpaired.fastq  data/trimmed/{f}_trimmed_1_unpaired.fastq ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36"
    proc_shell = sp.call(trim_call,shell=True)
    


TrimmomaticPE: Started with arguments:
 -threads 2 data/raw_fastq/SRR13349122_1.fastq data/raw_fastq/SRR13349122_2.fastq data/trimmed/SRR13349122_trimmed_1.fastq data/trimmed/SRR13349122_trimmed_2.fastq data/trimmed/SRR13349122_trimmed_2_unpaired.fastq data/trimmed/SRR13349122_trimmed_1_unpaired.fastq ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
Using PrefixPair: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT' and 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
ILLUMINACLIP: Using 1 prefix pairs, 0 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Read Pairs: 10827590 Both Surviving: 10810267 (99.84%) Forward Only Surviving: 17297 (0.16%) Reverse Only Surviving: 0 (0.00%) Dropped: 26 (0.00%)
TrimmomaticPE: Completed successfully
TrimmomaticPE: Started with arguments:
 -threads 2 data/raw_fastq/SRR13349123_1.fastq data/raw_fastq/SRR13349123_2.fastq data/trimmed/SRR13349123_trimmed_1.fastq data/trimmed/

In [28]:
# Let's align reads to the reference genome
# Asign Ref variable
REF='data/reference/M_chelonae_NZ_CP007220.fasta'
for f in files:
    bwa_call = f"bwa mem {REF} data/trimmed/{f}_trimmed_1.fastq  data/trimmed/{f}_trimmed_2.fastq > data/aligned/{f}.sam"
    proc_shell = sp.call(bwa_call,shell=True)

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[W::bseq_read] the 2nd file has fewer sequences.
[M::process] read 34594 sequences (1754537 bp)...
[W::bseq_read] the 2nd file has fewer sequences.
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (7188, 1, 1, 7203)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (503, 1139, 2019)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 5051)
[M::mem_pestat] mean and std.dev: (1361.19, 1035.94)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 6567)
[M::mem_pestat] skip orientation FR as there are not enough pairs
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation RR...
[M::mem_pestat] (25, 50, 75) percentile: (494, 1124, 2016)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 5060)
[M::mem_pestat] mean and std.dev: (1361.46

In [22]:
# Convert our sam file to binary form (bam)
for f in files:
    samtools_call = f"samtools view -S -b data/aligned/{f}.sam > data/aligned/{f}.bam"
    proc_shell = sp.call(samtools_call,shell=True)
# Remove the sam files
for f in os.listdir('./data/aligned'):
        if f.endswith('.sam'):
            os.remove(f)


FileNotFoundError: [Errno 2] No such file or directory: 'SRR13349122.sam'

['binder', 'pangeo-test.ipynb', 'README.md', 'scripts', 'TruSeq3-PE.fa', '.ipynb_checkpoints', 'data', 'SRR13349122.bam', 'SRR13349123.bam']
